In [ ]:
#main code
import random
import pandas as pd
import time
import csv

# Define days and hours
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
hours = ['8:00 AM', '9:00 AM', '10:00 AM', '11:00 AM', '12:00 PM', '1:00 PM', '2:00 PM', '3:00 PM','4:00 PM','5:00 PM','6:00 PM']

# Define the filename
filename = '/content/schedule.csv'

# Read the data from the CSV file
with open(filename, 'r') as file:
    reader = csv.DictReader(file)
    schedule_data = list(reader)

# Define lab hours and their capacities for first year
first_year_lab_hours = ['10:00 AM', '11:00 AM', '12:00 PM']
first_year_lab_capacities = {
    'Lab 1': 30,
    'Lab 2':40,
    'Lab 3': 35
}

# Define lab hours and their capacities for second year
second_year_lab_hours = ['10:00 AM', '11:00 AM', '12:00 PM']
second_year_lab_capacities = {
    'Lab 4': 30,
    'Lab 5': 25,
    'Lab 6': 35
}

# Define lunch hours for first year
first_year_lunch_hours = ['1:00 PM']

# Define lunch hours for second year
second_year_lunch_hours = ['1:00 PM','2:00 PM']

# Define office hours for first year
first_year_office_hours = ['2:00 PM']

# Define office hours for second year
second_year_office_hours = ['2:00 PM']

# Define short break hours
first_year_short_break_hours = ['4:00 PM']
second_year_short_break_hours = ['4:00 PM']

def fitness_function(schedule):
    conflicts = 0
    subjects = set()
    teachers = set()
    classrooms_booked = set()
    labs_booked = set()
    prev_subject = ''
    lab_count = 0

    for slot in schedule:
        day, hour, subject, teacher, location = slot

        if subject in subjects or teacher in teachers or location in classrooms_booked or (subject == 'Lab' and location in labs_booked):
            conflicts += 1

        if subject != 'No Subject':
            subjects.add(subject)
            teachers.add(teacher)
            if subject != 'Lab':
                classrooms_booked.add(location)
            else:
                labs_booked.add(location)
                lab_count += 1

        if subject != 'No Subject' and subject != 'Office Hours' and prev_subject != 'No Subject':
            if subject != prev_subject and hour not in first_year_short_break_hours or hour not in second_year_short_break_hours:
                conflicts += 1

        prev_subject = subject

    fitness_score = 1.0 / (conflicts + 1)
    return conflicts, fitness_score


# Calculate workload balance
def calculate_workload_balance(teachers):
    unique_teachers = set(teachers.values())
    max_workload = max(list(teachers.values()).count(teacher) for teacher in unique_teachers)
    min_workload = min(list(teachers.values()).count(teacher) for teacher in unique_teachers)
    workload_balance = max_workload - min_workload
    return workload_balance


# Calculate preference penalties
def calculate_preference_penalties(schedule):
    penalties = 0
    for slot in schedule:
        day, hour, subject, teacher, location = slot
        if subject in first_year_subjects and teacher != first_year_subjects[subject]:
            penalties += 1
        if subject in second_year_subjects and teacher != second_year_subjects[subject]:
            penalties += 1
    return penalties




def create_population(size):
    population = []
    for _ in range(size):
        schedule = []
        lab_counts = {
            'Lab 1': 0,
            'Lab 2': 0,
            'Lab 3': 0,
            'Lab 4': 0,
            'Lab 5': 0,
            'Lab 6': 0
        }

        # First Year Schedule
        for day in days:
            for hour in hours:
                if hour in first_year_lunch_hours and day in days[:5]:
                    subject = 'No Subject'
                    teacher = ''
                    location = ''
                elif hour in first_year_office_hours and day in days[:5]:
                    subject = 'Office Hours'
                    teacher = ''
                    location = 'Faculty Office'
                elif hour in first_year_short_break_hours and day in days[:5]:
                    subject = 'Short Break'
                    teacher = ''
                    location = ''
                else:
                    if day in days[:5] and hour not in first_year_lunch_hours and hour not in first_year_office_hours and hour not in first_year_short_break_hours:
                        if hour in first_year_lab_hours:
                            # Check lab count for the day
                            if lab_counts['Lab 1'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 1'
                                lab_counts['Lab 1'] += 1
                            elif lab_counts['Lab 2'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 2'
                                lab_counts['Lab 2'] += 1
                            elif lab_counts['Lab 3'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 3'
                                lab_counts['Lab 3'] += 1
                            else:
                                # If all labs are booked, assign a random subject
                                subject = random.choice(list(first_year_subjects.keys()))
                                teacher = first_year_subjects[subject]
                                location = first_year_classrooms[subject]
                        else:
                            subject = random.choice(list(first_year_subjects.keys()))
                            teacher = first_year_subjects[subject]
                            location = first_year_classrooms[subject]
                    else:
                        subject = 'No Subject'
                        teacher = ''
                        location = ''
                schedule.append((day, hour, subject, teacher, location))

        # Second Year Schedule
        for day in days:
            for hour in hours:
                if hour in second_year_lunch_hours and day in days[:5]:
                    subject = 'No Subject'
                    teacher = ''
                    location = ''
                elif hour in second_year_office_hours and day in days[:5]:
                    subject = 'Office Hours'
                    teacher = ''
                    location = 'Faculty Office'
                elif hour in second_year_short_break_hours and day in days[:5]:
                    subject = 'Short Break'
                    teacher = ''
                    location = ''
                else:
                    if day in days[:5] and hour not in second_year_lunch_hours and hour not in second_year_office_hours and hour not in second_year_short_break_hours:
                        if hour in second_year_lab_hours:
                            # Check lab count for the day
                            if lab_counts['Lab 4'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 4'
                                lab_counts['Lab 4'] += 1
                            elif lab_counts['Lab 5'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 5'
                                lab_counts['Lab 5'] += 1
                            elif lab_counts['Lab 6'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 6'
                                lab_counts['Lab 6'] += 1
                            else:
                                # If all labs are booked, assign a random subject
                                subject = random.choice(list(second_year_subjects.keys()))
                                teacher = second_year_subjects[subject]
                                location = second_year_classrooms[subject]
                        else:
                            subject = random.choice(list(second_year_subjects.keys()))
                            teacher = second_year_subjects[subject]
                            location = second_year_classrooms[subject]
                    else:
                        subject = 'No Subject'
                        teacher = ''
                        location = ''
                schedule.append((day, hour, subject, teacher, location))

        population.append(schedule)

    return population




# Select parents based on tournament selection
def select_parents(population, tournament_size):
    parents = []
    for _ in range(2):
        tournament = random.sample(population, tournament_size)
        best_schedule = min(tournament, key=lambda x: (fitness_function(x)[0], fitness_function(x)[1]))
        parents.append(best_schedule)
    return parents


# Perform crossover
def crossover(parents):
    parent1, parent2 = parents
    child = []
    for i in range(len(parent1)):
        if random.random() < 0.6:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child



def mutate(schedule):
    mutated_schedule = schedule.copy()

    # Randomly select a day and hour to mutate
    day_index = random.randint(0, len(days) - 1)
    hour_index = random.randint(0, len(hours) - 1)

    day, hour, subject, teacher, location = mutated_schedule[day_index * len(hours) + hour_index]

    # Skip mutation if it's a lunch hour, office hour, or short break
    if hour in first_year_lunch_hours or hour in second_year_lunch_hours or hour in first_year_office_hours or hour in second_year_office_hours or hour in first_year_short_break_hours or hour in second_year_short_break_hours:
        return mutated_schedule

    # Determine the year based on the selected day
    if day in days[:5]:  # First year
        # Mutate the subject
        if hour in first_year_lab_hours:
            subject = 'Lab'
            location = random.choice(list(first_year_lab_capacities.keys()))
        else:
            subject = random.choice(list(first_year_subjects.keys()))
            teacher = first_year_subjects[subject]
            location = first_year_classrooms[subject]
    else:  # Second year
        # Mutate the subject
        if hour in second_year_lab_hours:
            subject = 'Lab'
            location = random.choice(list(second_year_lab_capacities.keys()))
        else:
            subject = random.choice(list(second_year_subjects.keys()))
            teacher = second_year_subjects[subject]
            location = second_year_classrooms[subject]

    # Check for double lectures repetition
    if subject != 'Lab':
        for slot in mutated_schedule:
            if slot[2] == subject and slot[3] == teacher:
                return schedule

    # Check lab scheduling
    if subject == 'Lab':
        lab_count = sum(1 for slot in mutated_schedule if slot[2] == 'Lab' and slot[3] == teacher)
        if lab_count >= 3:
            return schedule

    mutated_schedule[day_index * len(hours) + hour_index] = (day, hour, subject, teacher, location)

    conflicts, fitness_score = fitness_function(mutated_schedule)
    current_conflicts, current_fitness_score = fitness_function(schedule)

    # Keep the mutated schedule only if it improves the conflicts or fitness score
    if conflicts < current_conflicts or fitness_score > current_fitness_score:
        return mutated_schedule
    else:
        return schedule





# Run the genetic algorithm with performance analysis
def genetic_algorithm(population_size, generations, tournament_size):
    population = create_population(population_size)
    best_conflicts = float('inf')
    best_fitness_score = float('inf')
    best_schedule = None
    start_time = time.time()  # Start tracking execution time
    for generation in range(generations):
        parents = select_parents(population, tournament_size)
        child = crossover(parents)
        child = mutate(child)
        population.append(child)
        population = sorted(population, key=lambda x: fitness_function(x)[0])
        population = population[:population_size]
        conflicts, fitness_score = fitness_function(population[0])
        if conflicts < best_conflicts:
            best_conflicts = conflicts
            best_fitness_score = fitness_score
            best_schedule = population[0]
        print(f"Generation: {generation + 1}")
        print(f"Best Conflicts: {best_conflicts}")
        print(f"Best Fitness Score: {best_fitness_score}")

    end_time = time.time()  # Stop tracking execution time
    print("Best Schedule:")
    print_schedule(best_schedule)

    # Print the best fitness score and best conflicts
    print(f"Final Best Fitness Score: {best_fitness_score}")
    print(f"Final Best Conflicts: {best_conflicts}")

    execution_time = end_time - start_time
    print(f"Execution Time: {execution_time} seconds")

# Display the schedule
def print_schedule(best_schedule):
    # Create separate dictionaries for the first year and second year timetables
    first_year_timetable = {}
    second_year_timetable = {}

    # Populate the timetables for each year
    for i, slot in enumerate(best_schedule):
        day, hour, subject, teacher, location = slot

        # Check the year based on the index
        year_timetable = first_year_timetable if i < len(best_schedule) // 2 else second_year_timetable

        # Check if it's a lunch hour
        if hour in first_year_lunch_hours or hour in second_year_lunch_hours:
            subject = "Lunch Break"

        # Check if it's an office hour
        if hour in first_year_office_hours or hour in second_year_office_hours:
            subject = "Office Hours"
            teacher = ""
            location = "Faculty Office"

        # Check if it's a short break
        if hour in first_year_short_break_hours or hour in second_year_short_break_hours:
            subject = "Short Break"

        if hour not in year_timetable:
            year_timetable[hour] = {}
        year_timetable[hour][day] = f"{subject}\n{teacher}\n{location}"

    # Convert the first year timetable dictionary to a DataFrame
    df_first_year = pd.DataFrame(first_year_timetable)

    # Save the first year timetable to Excel
    df_first_year.to_excel('timetable_first_year.xlsx')

    # Convert the second year timetable dictionary to a DataFrame
    df_second_year = pd.DataFrame(second_year_timetable)

    # Save the second year timetable to Excel
    df_second_year.to_excel('timetable_second_year.xlsx')

# Run the genetic algorithm
genetic_algorithm(population_size=1000, generations=100, tournament_size=3)








TypeError: ignored

In [ ]:
import csv

# Define the filename
filename = 'schedule.csv'

# Define the fieldnames for the CSV file
fieldnames = ['Subject', 'Teacher', 'Classroom', 'Capacity']

# Create a list of dictionaries for the schedule data
schedule_data = []

# First-year subjects and classrooms
first_year_subjects = {
    "Data Structures": "Shobhana Patil",
    "Digital Logic Design": "Smita Pawar",
    "Design Thinking and Fabrication Workshop": "Dr. Swapnil",
    "Applied Computational Mathematics": "Dr. Rishikant/Dr. Anju",
    "Human Physiology": "Reema Deshmukh"
}

first_year_classrooms = {
    'Data Structures': 'Room 101',
    'Digital Logic Design': 'Room 102',
    'Design Thinking and Fabrication Workshop': 'Room 103',
    'Applied Computational Mathematics': 'Room 104',
    'Human Physiology': 'Room 105'
}

# Capacities for classrooms
capacities = {
    'Room 101': 50,
    'Room 102': 40,
    'Room 103': 60,
    'Room 104': 45,
    'Room 105': 55
}

# Add first-year subjects to the schedule data
for subject, teacher in first_year_subjects.items():
    classroom = first_year_classrooms.get(subject, '')
    capacity = capacities.get(classroom, '')
    schedule_data.append({
        'Subject': subject,
        'Teacher': teacher,
        'Classroom': classroom,
        'Capacity': capacity
    })

# Second-year subjects and classrooms
second_year_subjects = {
    "Introduction to Communication Systems": "Dr. Gaurav Singh",
    "Introduction to Intelligent Systems": "Dr. Bandala",
    "Database Systems": "Mrs. Sandhya Jawale",
    "System Software": "Dr. Ashvini Shende",
    "Computer Networks": "Dr. Maheshwari",
    "Environmental Studies/Climate/Sustainable Development": "Navneet Agarwal"
}

second_year_classrooms = {
    'Introduction to Communication Systems': 'Room 201',
    'Introduction to Intelligent Systems': 'Room 202',
    'Database Systems': 'Room 203',
    'System Software': 'Room 204',
    'Computer Networks': 'Room 205',
    'Environmental Studies/Climate/Sustainable Development': 'Room 206'
}

# Capacities for classrooms
capacities = {
    'Room 201': 30,
    'Room 202': 35,
    'Room 203': 25,
    'Room 204': 40,
    'Room 205': 50,
    'Room 206': 60
}

# Add second-year subjects to the schedule data
for subject, teacher in second_year_subjects.items():
    classroom = second_year_classrooms.get(subject, '')
    capacity = capacities.get(classroom, '')
    schedule_data.append({
        'Subject': subject,
        'Teacher': teacher,
        'Classroom': classroom,
        'Capacity': capacity
    })



# third-year subjects and classrooms
third_year_cybersecuritysubjects = {
    "Deep Neural Network":	"Dr.Shwet Chandan",
     "System Software Security":	"Dr.Somya Dubey",
     "Computer Forensics":	"Pulkit Tiwari",
     "Marketing Modelling":	"Mrs.Sheetal B",
     "Finance Risk Management":"Dr.Palash B",
     "Fintech Regulation":"Dr: Priyanka Dhoot"
}

third_year_cybersecurityclassrooms = {
    'Deep Neural Network': 'Room 301',
    'System Software Security': 'Room 302',
    'Computer Forensics': 'Room 303',
    'System Software': 'Room 304',
    'Marketing Modelling': 'Room 305',
    'Finance Risk Management': 'Room 306',
    'Fintech Regulation':'Room 307'
}

capacities = {
    'Room 301': 120,
    'Room 302': 120,
    'Room 303': 120,
    'Room 304': 120,
    'Room 305': 120,
    'Room 306': 120,
    'Room 307': 120

}



# Add third-year cssubjects to the schedule data
for subject, teacher in third_year_cybersecuritysubjects.items():
    classroom = third_year_cybersecurityclassrooms.get(subject, '')
    capacity = capacities.get(classroom, '')
    schedule_data.append({
        'Subject': subject,
        'Teacher': teacher,
        'Classroom': classroom,
        'Capacity': capacity

    })

# third-year subjects and classrooms
third_year_aimlsubjects = {
    "Deep Neural Network":"Dr.Shwet Chandan",
     "Game theory":	"Dr.Kapil",
     "High performance computing":"Mr.Pulkit Tiwari",
      "Marketing Modelling":	"Mrs.Sheetal B",
      "Finance Risk Management"	:	"Dr.Palash B",
       "Fintech Regulation"	:	"Dr.Priyanka Dhoot"
}

third_year_aimlclassrooms = {
    "Deep Neural Network":"Room 401",
     "Game theory":	"Room 402",
     "High performance computing":"Room 403",
      "Marketing Modelling":	"Room 404",
      "Finance Risk Management"	:	"Room 405",
       "Fintech Regulation"	:	"Room 406"
}

capacities = {
    'Room 401': 120,
    'Room 402': 120,
    'Room 403': 120,
    'Room 404': 120,
    'Room 405': 120,
    'Room 406': 120

}


# Add third-year cssubjects to the schedule data
for subject, teacher in third_year_aimlsubjects.items():
    classroom = third_year_aimlclassrooms.get(subject, '')
    capacity = capacities.get(classroom, '')
    schedule_data.append({
        'Subject': subject,
        'Teacher': teacher,
        'Classroom': classroom,
        'Capacity': capacity

    })




# third-year subjects and classrooms
third_year_dssubjects = {
"Deep Neural Network"	:	"Dr.Shwet Chandan",
"Big Data Analytics"		:"Dr.Samrjit Roy",
"Knowledge discovery & data mining":"Dr.Rahul Sharma",
"Marketing Modelling"	:"Mrs.Sheetal B",
"Finance Risk Management":	"Dr.Palash B",
"Fintech Regulation":"Dr.Priyanka Dhoot"
}

third_year_dsclassrooms = {
"Deep Neural Network"	:	"Room 501",
"Big Data Analytics"		:"Room 502 ",
"Knowledge discovery & data mining":"Room 503",
"Marketing Modelling"	:"Room 504",
"Finance Risk Management":	"Room 505",
"Fintech Regulation":"Room 506"
}

capacities = {
    'Room 501': 120,
    'Room 502': 120,
    'Room 503': 120,
    'Room 504': 120,
    'Room 505': 120,
    'Room 506': 120

}

# Add third-year cssubjects to the schedule data
for subject, teacher in third_year_dssubjects.items():
    classroom = third_year_dsclassrooms.get(subject, '')
    capacity = capacities.get(classroom, '')
    schedule_data.append({
        'Subject': subject,
        'Teacher': teacher,
        'Classroom': classroom,
        'Capacity': capacity

    })

    # fourth-year subjects and classrooms
fourth_year_subjects = {
"Fintech"	:	"Dr. Vinayak Sathe",
"Block chain "		:"Sulaxan Jadhav",
"UI&UX ":"Varun sir",
"Advanced Data Science"	:"Mrs.Sheetal B",
"Behavioural Finance":	"Dr.Palash B",
"Digital Communication":"Dr.Priyanka Dhoot"
}

fourth_year_classrooms = {
"Fintech"	:	"Room 601",
"Block chain "		:"Room 602",
"UI&UX ":"Room 603",
"Advanced Data Science"	:"Room 604",
"Behavioural Finance":	"Room 605",
"Digital Communication":"Room 606"
}

capacities = {
    'Room 601': 120,
    'Room 602': 120,
    'Room 603': 120,
    'Room 604': 120,
    'Room 605': 120,
    'Room 606': 120

}

# Add third-year cssubjects to the schedule data
for subject, teacher in fourth_year_subjects.items():
    classroom = fourth_year_classrooms.get(subject, '')
    capacity = capacities.get(classroom, '')
    schedule_data.append({
        'Subject': subject,
        'Teacher': teacher,
        'Classroom': classroom,
        'Capacity': capacity

    })

# Write the data to the CSV file
with open(filename, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(schedule_data)



In [ ]:
#aiml & ds
import random
import pandas as pd
import time
import csv

# Define days and hours
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
hours = ['8:00 AM', '9:00 AM', '10:00 AM', '11:00 AM', '12:00 PM', '1:00 PM', '2:00 PM', '3:00 PM','4:00 PM','5:00 PM','6:00 PM']

# Define the filename
filename = '/content/schedule.csv'

# Read the data from the CSV file
with open(filename, 'r') as file:
    reader = csv.DictReader(file)
    schedule_data = list(reader)

# Define lab hours and their capacities for first year
third_year_aimllab_hours = ['8:00 AM', '9:00 AM', '10:00 AM']
third_year_aimllab_capacities = {
    'Lab 1': 30,
    'Lab 2':40,
    'Lab 3': 35
}

# Define lab hours and their capacities for second year
third_year_dslab_hours = ['8:00 AM', '9:00 AM', '10:00 AM']
third_year_dslab_capacities = {
    'Lab 4': 30,
    'Lab 5': 25,
    'Lab 6': 35
}

# Define lunch hours for first year
third_year_aimllunch_hours = ['1:00 PM']

# Define lunch hours for second year
third_year_dslunch_hours = ['12:00 PM','1:00 PM']

# Define office hours for first year
third_year_aimloffice_hours = ['2:00 PM']

# Define office hours for second year
third_year_dsoffice_hours = ['4:30 PM']

# Define short break hours
third_year_aimlshort_break_hours = ['4:00 PM']
third_year_dsshort_break_hours = ['4:00 PM']

def fitness_function(schedule):
    conflicts = 0
    subjects = set()
    teachers = set()
    classrooms_booked = set()
    labs_booked = set()
    prev_subject = ''
    lab_count = 0

    for slot in schedule:
        day, hour, subject, teacher, location = slot

        if subject in subjects or teacher in teachers or location in classrooms_booked or (subject == 'Lab' and location in labs_booked):
            conflicts += 1

        if subject != 'No Subject':
            subjects.add(subject)
            teachers.add(teacher)
            if subject != 'Lab':
                classrooms_booked.add(location)
            else:
                labs_booked.add(location)
                lab_count += 1

        if subject != 'No Subject' and subject != 'Office Hours' and prev_subject != 'No Subject':
            if subject != prev_subject and hour not in third_year_aimlshort_break_hours or hour not in third_year_dsshort_break_hours:
                conflicts += 1

        prev_subject = subject

    fitness_score = 1.0 / (conflicts + 1)
    return conflicts, fitness_score


# Calculate workload balance
def calculate_workload_balance(teachers):
    unique_teachers = set(teachers.values())
    max_workload = max(list(teachers.values()).count(teacher) for teacher in unique_teachers)
    min_workload = min(list(teachers.values()).count(teacher) for teacher in unique_teachers)
    workload_balance = max_workload - min_workload
    return workload_balance


# Calculate preference penalties
def calculate_preference_penalties(schedule):
    penalties = 0
    for slot in schedule:
        day, hour, subject, teacher, location = slot
        if subject in third_year_aimlsubjects and teacher != third_year_aimlsubjects[subject]:
            penalties += 1
        if subject in third_year_dssubjects and teacher != third_year_dssubjects[subject]:
            penalties += 1
    return penalties




def create_population(size):
    population = []
    for _ in range(size):
        schedule = []
        lab_counts = {
            'Lab 1': 0,
            'Lab 2': 0,
            'Lab 3': 0,
            'Lab 4': 0,
            'Lab 5': 0,
            'Lab 6': 0
        }

        # Third Year AIML Schedule
        for day in days:
            for hour in hours:
                if hour in third_year_aimllunch_hours and day in days[:5]:
                    subject = 'No Subject'
                    teacher = ''
                    location = ''
                elif hour in third_year_aimloffice_hours and day in days[:5]:
                    subject = 'Office Hours'
                    teacher = ''
                    location = 'Faculty Office'
                elif hour in third_year_aimlshort_break_hours and day in days[:5]:
                    subject = 'Short Break'
                    teacher = ''
                    location = ''
                else:
                    if day in days[:5] and hour not in third_year_aimllunch_hours and hour not in third_year_aimloffice_hours and hour not in third_year_aimlshort_break_hours:
                        if hour in third_year_aimllab_hours:
                            # Check lab count for the day
                            if lab_counts['Lab 1'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 1'
                                lab_counts['Lab 1'] += 1
                            elif lab_counts['Lab 2'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 2'
                                lab_counts['Lab 2'] += 1
                            elif lab_counts['Lab 3'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 3'
                                lab_counts['Lab 3'] += 1
                            else:
                                # If all labs are booked, assign a random subject
                                subject = random.choice(list(third_year_aimlsubjects.keys()))
                                teacher = third_year_aimlsubjects[subject]
                                location = third_year_aimlclassrooms[subject]
                        else:
                            subject = random.choice(list(third_year_aimlsubjects.keys()))
                            teacher = third_year_aimlsubjects[subject]
                            location = third_year_aimlclassrooms[subject]
                    else:
                        subject = 'No Subject'
                        teacher = ''
                        location = ''
                schedule.append((day, hour, subject, teacher, location))

        # thirs Year ds Schedule
        for day in days:
            for hour in hours:
                if hour in third_year_dslunch_hours and day in days[:5]:
                    subject = 'No Subject'
                    teacher = ''
                    location = ''
                elif hour in third_year_dsoffice_hours and day in days[:5]:
                    subject = 'Office Hours'
                    teacher = ''
                    location = 'Faculty Office'
                elif hour in third_year_dsshort_break_hours and day in days[:5]:
                    subject = 'Short Break'
                    teacher = ''
                    location = ''
                else:
                    if day in days[:5] and hour not in third_year_dslunch_hours and hour not in third_year_dsoffice_hours and hour not in third_year_dsshort_break_hours:
                        if hour in third_year_dslab_hours:
                            # Check lab count for the day
                            if lab_counts['Lab 4'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 4'
                                lab_counts['Lab 4'] += 1
                            elif lab_counts['Lab 5'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 5'
                                lab_counts['Lab 5'] += 1
                            elif lab_counts['Lab 6'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 6'
                                lab_counts['Lab 6'] += 1
                            else:
                                # If all labs are booked, assign a random subject
                                subject = random.choice(list(third_year_dssubjects.keys()))
                                teacher = third_year_dssubjects[subject]
                                location = third_year_dsclassrooms[subject]
                        else:
                            subject = random.choice(list(third_year_dssubjects.keys()))
                            teacher = third_year_dssubjects[subject]
                            location = third_year_dsclassrooms[subject]
                    else:
                        subject = 'No Subject'
                        teacher = ''
                        location = ''
                schedule.append((day, hour, subject, teacher, location))

        population.append(schedule)

    return population




# Select parents based on tournament selection
def select_parents(population, tournament_size):
    parents = []
    for _ in range(2):
        tournament = random.sample(population, tournament_size)
        best_schedule = min(tournament, key=lambda x: (fitness_function(x)[0], fitness_function(x)[1]))
        parents.append(best_schedule)
    return parents


# Perform crossover
def crossover(parents):
    parent1, parent2 = parents
    child = []
    for i in range(len(parent1)):
        if random.random() < 0.6:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child



def mutate(schedule):
    mutated_schedule = schedule.copy()

    # Randomly select a day and hour to mutate
    day_index = random.randint(0, len(days) - 1)
    hour_index = random.randint(0, len(hours) - 1)

    day, hour, subject, teacher, location = mutated_schedule[day_index * len(hours) + hour_index]

    # Skip mutation if it's a lunch hour, office hour, or short break
    if hour in third_year_aimllunch_hours or hour in third_year_dslunch_hours or hour in third_year_aimloffice_hours or hour in third_year_dsoffice_hours or hour in third_year_aimlshort_break_hours or hour in third_year_dsshort_break_hours:
        return mutated_schedule

    # Determine the year based on the selected day
    if day in days[:5]:  # First year
        # Mutate the subject
        if hour in third_year_aimllab_hours:
            subject = 'Lab'
            location = random.choice(list(third_year_aimllab_capacities.keys()))
        else:
            subject = random.choice(list(third_year_aimlsubjects.keys()))
            teacher = third_year_aimlsubjects[subject]
            location = third_year_aimlclassrooms[subject]
    else:  # Third Ds year
        # Mutate the subject
        if hour in third_year_dslab_hours:
            subject = 'Lab'
            location = random.choice(list(third_year_dslab_capacities.keys()))
        else:
            subject = random.choice(list(third_year_dssubjects.keys()))
            teacher = third_year_dssubjects[subject]
            location = third_year_dsclassrooms[subject]

    # Check for double lectures repetition
    if subject != 'Lab':
        for slot in mutated_schedule:
            if slot[2] == subject and slot[3] == teacher:
                return schedule

    # Check lab scheduling
    if subject == 'Lab':
        lab_count = sum(1 for slot in mutated_schedule if slot[2] == 'Lab' and slot[3] == teacher)
        if lab_count >= 3:
            return schedule

    mutated_schedule[day_index * len(hours) + hour_index] = (day, hour, subject, teacher, location)

    conflicts, fitness_score = fitness_function(mutated_schedule)
    current_conflicts, current_fitness_score = fitness_function(schedule)

    # Keep the mutated schedule only if it improves the conflicts or fitness score
    if conflicts < current_conflicts or fitness_score > current_fitness_score:
        return mutated_schedule
    else:
        return schedule





# Run the genetic algorithm with performance analysis
def genetic_algorithm(population_size, generations, tournament_size):
    population = create_population(population_size)
    best_conflicts = float('inf')
    best_fitness_score = float('inf')
    best_schedule = None
    start_time = time.time()  # Start tracking execution time
    for generation in range(generations):
        parents = select_parents(population, tournament_size)
        child = crossover(parents)
        child = mutate(child)
        population.append(child)
        population = sorted(population, key=lambda x: fitness_function(x)[0])
        population = population[:population_size]
        conflicts, fitness_score = fitness_function(population[0])
        if conflicts < best_conflicts:
            best_conflicts = conflicts
            best_fitness_score = fitness_score
            best_schedule = population[0]
        print(f"Generation: {generation + 1}")
        print(f"Best Conflicts: {best_conflicts}")
        print(f"Best Fitness Score: {best_fitness_score}")

    end_time = time.time()  # Stop tracking execution time
    print("Best Schedule:")
    print_schedule(best_schedule)

    # Print the best fitness score and best conflicts
    print(f"Final Best Fitness Score: {best_fitness_score}")
    print(f"Final Best Conflicts: {best_conflicts}")

    execution_time = end_time - start_time
    print(f"Execution Time: {execution_time} seconds")

# Display the schedule
def print_schedule(best_schedule):
    # Create separate dictionaries for the first year and second year timetables
    third_year_aimltimetable = {}
    third_year_dstimetable = {}

    # Populate the timetables for each year
    for i, slot in enumerate(best_schedule):
        day, hour, subject, teacher, location = slot

        # Check the year based on the index
        year_timetable = third_year_aimltimetable if i < len(best_schedule) // 2 else third_year_dstimetable

        # Check if it's a lunch hour
        if hour in third_year_aimllunch_hours or hour in third_year_dslunch_hours:
            subject = "Lunch Break"

        # Check if it's an office hour
        if hour in third_year_aimloffice_hours or hour in third_year_dsoffice_hours:
            subject = "Office Hours"
            teacher = ""
            location = "Faculty Office"

        # Check if it's a short break
        if hour in third_year_aimlshort_break_hours or hour in third_year_dsshort_break_hours:
            subject = "Short Break"

        if hour not in year_timetable:
            year_timetable[hour] = {}
        year_timetable[hour][day] = f"{subject}\n{teacher}\n{location}"

    # Convert the first year timetable dictionary to a DataFrame
    df_third_year_aiml = pd.DataFrame(third_year_aimltimetable)

    # Save the first year timetable to Excel
    df_third_year_aiml.to_excel('timetable_third_year_aiml.xlsx')

    # Convert the second year timetable dictionary to a DataFrame
    df_third_year_ds = pd.DataFrame(third_year_dstimetable)

    # Save the second year timetable to Excel
    df_third_year_ds.to_excel('timetable_third_year_ds.xlsx')

# Run the genetic algorithm
genetic_algorithm(population_size=1000, generations=1, tournament_size=2)

Generation: 1
Best Conflicts: 176
Best Fitness Score: 0.005649717514124294
Best Schedule:
Final Best Fitness Score: 0.005649717514124294
Final Best Conflicts: 176
Execution Time: 0.04876136779785156 seconds


In [ ]:
#cybersecurtiy & fourth year
import random
import pandas as pd
import time
import csv

# Define days and hours
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
hours = ['8:00 AM', '9:00 AM', '10:00 AM', '11:00 AM', '12:00 PM', '1:00 PM', '2:00 PM', '3:00 PM','4:00 PM','5:00 PM','6:00 PM']

# Define the filename
filename = '/content/schedule.csv'

# Read the data from the CSV file
with open(filename, 'r') as file:
    reader = csv.DictReader(file)
    schedule_data = list(reader)

# Define lab hours and their capacities for first year
third_year_cybersecuritylab_hours = ['2:00 PM', '3:00 PM', '4:00 PM']
third_year_cybersecuritylab_capacities = {
    'Lab 1': 30,
    'Lab 2':40,
    'Lab 3': 35
}

# Define lab hours and their capacities for second year
fourth_year_lab_hours = [ '2:00 PM', '3:00 PM']
fourth_year_lab_capacities = {
    'Lab 4': 30,
    'Lab 5': 25,
    'Lab 6': 35
}

# Define lunch hours for first year
third_year_cybersecuritylunch_hours = ['1:00 PM']

# Define lunch hours for second year
fourth_year_lunch_hours = ['1:00 PM']

# Define office hours for third year cybersecurity year
third_year_cybersecurityoffice_hours = ['11:00 AM']

# Define office hours for fourth year
fourth_year_office_hours = ['11:00 PM']

# Define short break hours
third_year_cybersecurityshort_break_hours = ['4:00 PM']
fourth_year_short_break_hours = ['4:00 PM']

def fitness_function(schedule):
    conflicts = 0
    subjects = set()
    teachers = set()
    classrooms_booked = set()
    labs_booked = set()
    prev_subject = ''
    lab_count = 0

    for slot in schedule:
        day, hour, subject, teacher, location = slot

        if subject in subjects or teacher in teachers or location in classrooms_booked or (subject == 'Lab' and location in labs_booked):
            conflicts += 1

        if subject != 'No Subject':
            subjects.add(subject)
            teachers.add(teacher)
            if subject != 'Lab':
                classrooms_booked.add(location)
            else:
                labs_booked.add(location)
                lab_count += 1

        if subject != 'No Subject' and subject != 'Office Hours' and prev_subject != 'No Subject':
            if subject != prev_subject and hour not in third_year_cybersecurityshort_break_hours or hour not in fourth_year_short_break_hours:
                conflicts += 1

        prev_subject = subject

    fitness_score = 1.0 / (conflicts + 1)
    return conflicts, fitness_score


# Calculate workload balance
def calculate_workload_balance(teachers):
    unique_teachers = set(teachers.values())
    max_workload = max(list(teachers.values()).count(teacher) for teacher in unique_teachers)
    min_workload = min(list(teachers.values()).count(teacher) for teacher in unique_teachers)
    workload_balance = max_workload - min_workload
    return workload_balance


# Calculate preference penalties
def calculate_preference_penalties(schedule):
    penalties = 0
    for slot in schedule:
        day, hour, subject, teacher, location = slot
        if subject in third_year_cybersecuritysubjects and teacher != third_year_cybersecuritysubjects[subject]:
            penalties += 1
        if subject in fourth_year_subjects and teacher != fourth_year_subjects[subject]:
            penalties += 1
    return penalties




def create_population(size):
    population = []
    for _ in range(size):
        schedule = []
        lab_counts = {
            'Lab 1': 0,
            'Lab 2': 0,
            'Lab 3': 0,
            'Lab 4': 0,
            'Lab 5': 0,
            'Lab 6': 0
        }

        # Third Year cybersecurity Schedule
        for day in days:
            for hour in hours:
                if hour in third_year_cybersecuritylunch_hours and day in days[:5]:
                    subject = 'No Subject'
                    teacher = ''
                    location = ''
                elif hour in third_year_cybersecurityoffice_hours and day in days[:5]:
                    subject = 'Office Hours'
                    teacher = ''
                    location = 'Faculty Office'
                elif hour in third_year_cybersecurityshort_break_hours and day in days[:5]:
                    subject = 'Short Break'
                    teacher = ''
                    location = ''
                else:
                    if day in days[:5] and hour not in third_year_cybersecuritylunch_hours and hour not in third_year_cybersecurityoffice_hours and hour not in third_year_cybersecurityshort_break_hours:
                        if hour in third_year_cybersecuritylab_hours:
                            # Check lab count for the day
                            if lab_counts['Lab 1'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 1'
                                lab_counts['Lab 1'] += 1
                            elif lab_counts['Lab 2'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 2'
                                lab_counts['Lab 2'] += 1
                            elif lab_counts['Lab 3'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 3'
                                lab_counts['Lab 3'] += 1
                            else:
                                # If all labs are booked, assign a random subject
                                subject = random.choice(list(third_year_cybersecuritysubjects.keys()))
                                teacher = third_year_cybersecuritysubjects[subject]
                                location = third_year_cybersecurityclassrooms[subject]
                        else:
                            subject = random.choice(list(third_year_cybersecuritysubjects.keys()))
                            teacher = third_year_cybersecuritysubjects[subject]
                            location = third_year_cybersecurityclassrooms[subject]
                    else:
                        subject = 'No Subject'
                        teacher = ''
                        location = ''
                schedule.append((day, hour, subject, teacher, location))

        # fourth Year  Schedule
        for day in days:
            for hour in hours:
                if hour in fourth_year_lunch_hours and day in days[:5]:
                    subject = 'No Subject'
                    teacher = ''
                    location = ''
                elif hour in fourth_year_office_hours and day in days[:5]:
                    subject = 'Office Hours'
                    teacher = ''
                    location = 'Faculty Office'
                elif hour in fourth_year_short_break_hours and day in days[:5]:
                    subject = 'Short Break'
                    teacher = ''
                    location = ''
                else:
                    if day in days[:5] and hour not in fourth_year_lunch_hours and hour not in fourth_year_office_hours and hour not in fourth_year_short_break_hours:
                        if hour in fourth_year_lab_hours:
                            # Check lab count for the day
                            if lab_counts['Lab 4'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 4'
                                lab_counts['Lab 4'] += 1
                            elif lab_counts['Lab 5'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 5'
                                lab_counts['Lab 5'] += 1
                            elif lab_counts['Lab 6'] < 3:
                                subject = 'Lab'
                                teacher = ''
                                location = 'Lab 6'
                                lab_counts['Lab 6'] += 1
                            else:
                                # If all labs are booked, assign a random subject
                                subject = random.choice(list(fourth_year_subjects.keys()))
                                teacher = fourth_year_subjects[subject]
                                location = fourth_year_classrooms[subject]
                        else:
                            subject = random.choice(list(fourth_year_subjects.keys()))
                            teacher = fourth_year_subjects[subject]
                            location = fourth_year_classrooms[subject]
                    else:
                        subject = 'No Subject'
                        teacher = ''
                        location = ''
                schedule.append((day, hour, subject, teacher, location))

        population.append(schedule)

    return population




# Select parents based on tournament selection
def select_parents(population, tournament_size):
    parents = []
    for _ in range(2):
        tournament = random.sample(population, tournament_size)
        best_schedule = min(tournament, key=lambda x: (fitness_function(x)[0], fitness_function(x)[1]))
        parents.append(best_schedule)
    return parents


# Perform crossover
def crossover(parents):
    parent1, parent2 = parents
    child = []
    for i in range(len(parent1)):
        if random.random() < 0.6:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child



def mutate(schedule):
    mutated_schedule = schedule.copy()

    # Randomly select a day and hour to mutate
    day_index = random.randint(0, len(days) - 1)
    hour_index = random.randint(0, len(hours) - 1)

    day, hour, subject, teacher, location = mutated_schedule[day_index * len(hours) + hour_index]

    # Skip mutation if it's a lunch hour, office hour, or short break
    if hour in third_year_cybersecuritylunch_hours or hour in fourth_year_lunch_hours or hour in third_year_cybersecurityoffice_hours or hour in fourth_year_office_hours or hour in third_year_cybersecurityshort_break_hours or hour in fourth_year_short_break_hours:
        return mutated_schedule

    # Determine the year based on the selected day
    if day in days[:5]:  # third year cybersecurity
        # Mutate the subject
        if hour in third_year_cybersecuritylab_hours:
            subject = 'Lab'
            location = random.choice(list(third_year_cybersecuritylab_capacities.keys()))
        else:
            subject = random.choice(list(third_year_cybersecuritysubjects.keys()))
            teacher = third_year_cybersecuritysubjects[subject]
            location = third_year_cybersecurityclassrooms[subject]
    else:  # fouth year
        # Mutate the subject
        if hour in fourth_year_lab_hours:
            subject = 'Lab'
            location = random.choice(list(fourth_year_lab_capacities.keys()))
        else:
            subject = random.choice(list(fourth_year_subjects.keys()))
            teacher = fourth_year_subjects[subject]
            location = fourth_year_classrooms[subject]

    # Check for double lectures repetition
    if subject != 'Lab':
        for slot in mutated_schedule:
            if slot[2] == subject and slot[3] == teacher:
                return schedule

    # Check lab scheduling
    if subject == 'Lab':
        lab_count = sum(1 for slot in mutated_schedule if slot[2] == 'Lab' and slot[3] == teacher)
        if lab_count >= 3:
            return schedule

    mutated_schedule[day_index * len(hours) + hour_index] = (day, hour, subject, teacher, location)

    conflicts, fitness_score = fitness_function(mutated_schedule)
    current_conflicts, current_fitness_score = fitness_function(schedule)

    # Keep the mutated schedule only if it improves the conflicts or fitness score
    if conflicts < current_conflicts or fitness_score > current_fitness_score:
        return mutated_schedule
    else:
        return schedule





# Run the genetic algorithm with performance analysis
def genetic_algorithm(population_size, generations, tournament_size):
    population = create_population(population_size)
    best_conflicts = float('inf')
    best_fitness_score = float('inf')
    best_schedule = None
    start_time = time.time()  # Start tracking execution time
    for generation in range(generations):
        parents = select_parents(population, tournament_size)
        child = crossover(parents)
        child = mutate(child)
        population.append(child)
        population = sorted(population, key=lambda x: fitness_function(x)[0])
        population = population[:population_size]
        conflicts, fitness_score = fitness_function(population[0])
        if conflicts < best_conflicts:
            best_conflicts = conflicts
            best_fitness_score = fitness_score
            best_schedule = population[0]
        print(f"Generation: {generation + 1}")
        print(f"Best Conflicts: {best_conflicts}")
        print(f"Best Fitness Score: {best_fitness_score}")

    end_time = time.time()  # Stop tracking execution time
    print("Best Schedule:")
    print_schedule(best_schedule)

    # Print the best fitness score and best conflicts
    print(f"Final Best Fitness Score: {best_fitness_score}")
    print(f"Final Best Conflicts: {best_conflicts}")

    execution_time = end_time - start_time
    print(f"Execution Time: {execution_time} seconds")

# Display the schedule
def print_schedule(best_schedule):
    # Create separate dictionaries for the first year and second year timetables
    third_year_cybersecuritytimetable = {}
    fourth_year_timetable = {}

    # Populate the timetables for each year
    for i, slot in enumerate(best_schedule):
        day, hour, subject, teacher, location = slot

        # Check the year based on the index
        year_timetable = third_year_cybersecuritytimetable if i < len(best_schedule) // 2 else fourth_year_timetable

        # Check if it's a lunch hour
        if hour in third_year_cybersecuritylunch_hours or hour in fourth_year_lunch_hours:
            subject = "Lunch Break"

        # Check if it's an office hour
        if hour in third_year_cybersecurityoffice_hours or hour in fourth_year_office_hours:
            subject = "Office Hours"
            teacher = ""
            location = "Faculty Office"

        # Check if it's a short break
        if hour in third_year_cybersecurityshort_break_hours or hour in fourth_year_short_break_hours:
            subject = "Short Break"

        if hour not in year_timetable:
            year_timetable[hour] = {}
        year_timetable[hour][day] = f"{subject}\n{teacher}\n{location}"

    # Convert the third year cybersecurity timetable dictionary to a DataFrame
    df_third_year_cybersecurity = pd.DataFrame(third_year_cybersecuritytimetable)

    # Save the third year cybersecurity timetable to Excel
    df_third_year_cybersecurity.to_excel('timetable_third_year_cybersecurity.xlsx')

    # Convert the fourth year timetable dictionary to a DataFrame
    df_fourth_year = pd.DataFrame(fourth_year_timetable)

    # Save the fourth year timetable to Excel
    df_fourth_year.to_excel('timetable_fourth_year.xlsx')

# Run the genetic algorithm
genetic_algorithm(population_size=1000, generations=100, tournament_size=2)

Generation: 1
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 2
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 3
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 4
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 5
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 6
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 7
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 8
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 9
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 10
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 11
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 12
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 13
Best Conflicts: 174
Best Fitness Score: 0.005714285714285714
Generation: 14
Best C